In [1]:
import os
import base64
import io
from PIL import Image
import dotenv
from openai import AzureOpenAI


In [2]:
# Azure OpenAI configuration
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_api_version = os.getenv("OPENAI_API_VERSION", "2023-07-01-preview")
model = os.getenv("LLM_MODEL", "gpt-35-turbo")

print(f"Azure Endpoint: {azure_endpoint}")
print(f"Azure API Key available: {'Yes' if azure_api_key else 'No'}")
print(f"Azure API Version: {azure_api_version}")
print(f"Model deployment name: {model}")

# Initialize Azure OpenAI client if enabled

client = AzureOpenAI(
)

Azure Endpoint: https://aoai-sweden-505.openai.azure.com/
Azure API Key available: Yes
Azure API Version: 2025-03-01-preview
Model deployment name: gpt-4o


In [5]:
prompt = "What is in this image?"
img_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/2023_06_08_Raccoon1.jpg/1599px-2023_06_08_Raccoon1.jpg"

response = client.responses.create(
    model="gpt-4v",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt},
                # {"type": "input_image", "image_url": f"{img_url}"},
            ],
        }
    ],
)

In [16]:
response.output[0].content[0].text

"It seems you've mentioned an image, but since I can't view images directly, could you describe it to me? I'd be happy to help analyze or provide information based on your description!"

In [18]:
import requests


# Image file path
IMAGE_PATH = 'temp_image.png'


# Read the image as binary
with open(IMAGE_PATH, "rb") as image_file:
    image_data = image_file.read()

# Request headers and payload
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {azure_api_key}"
}

payload = {
    "model": "gpt-4o",
    "messages": [
        {"role": "system", "content": "Describe the image."},
        {"role": "user", "content": image_data.decode("latin-1")}
    ]
}

# Send the request to Azure OpenAI
response = requests.post(azure_endpoint, headers=headers, json=payload)

# Print the response
print(response.json())


{'error': {'code': '404', 'message': 'Resource not found'}}
